<a href="https://colab.research.google.com/github/zenon10/POC-OCR/blob/main/extract_text_v20230207.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Installations**

In [ ]:
## For pdf
! pip install pypdfium2

In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract

#Explicitly for french language
! sudo apt-get install tesseract-ocr-fra

In [ ]:
## For textract 
!apt-get install python-dev libxml2-dev libxslt1-dev antiword unrtf poppler-utils \
     pstotext tesseract-ocr \
     flac ffmpeg lame libmad0 libsox-fmt-mp3 sox libjpeg-dev swig libasound2-dev libpulse-dev
!pip install git+https://github.com/deanmalmgren/textract

##**Libraries**

In [ ]:
# Library to work with Operating Syatem
import os

# Library to create PDF, inspection, manupulation and rendering(turning into images)
import pypdfium2 as pdfium

# Regex lirary to manipulate string
import re

from PIL import Image

import textract

In [ ]:
import os
import glob
import pandas as pd
import pytesseract
import shutil
import os
import random
try:
    from PIL import Image
except ImportError:
    import Image

##**Defining Function**

In [ ]:
def pdf_to_img(input1):
    
    # if not os.path.exists(output_folder_path):
    #     os.makedirs(output_folder_path)

    input_file_name = os.path.split(input1)[-1].replace('.pdf','')   
    print('Extracting text from {}'.format(input_file_name))
    
    # PdfDocument (supports file path strings, bytes, and byte buffers)
    pdf_info = pdfium.PdfDocument(input1)
    
    # get the number of pages in the document
    n_pages = len(pdf_info)
    print(n_pages)
    
    page_indices = [i for i in range(n_pages)]    
    renderer = pdf_info.render_to(pdfium.BitmapConv.pil_image, page_indices = page_indices, scale = 300/72)

    page_number = 1
    output_folder_path = os.path.split(input1)[0]
    for images in renderer:
#         print(images.show())

        images.save(output_folder_path + "/" + str(page_number)  + '.jpeg')
        page_number +=1

In [160]:
def img_to_txt(img_path,language):
    
    # if not os.path.exists(output_folder_path):
    #     os.makedirs(output_folder_path)
    
    try:
        path = glob.glob(img_path + "/*.jpeg")
        print(path)
    except:
        print('image not in jpeg')
    
        
    ## Blank string to concatenate all text string outputs from ocr for each image
    combined_text = ''

    ## Defining output folder to save your text file
    output_folder_path = img_path
    # print(output_folder_path)
    for img in path:
        
        file_name = os.path.basename(img).replace('.jpeg','')
        print("File name:", file_name)
        
        ## Extracting text from image
        text = pytesseract.image_to_string(Image.open(img), lang=language)
        
        ## Concatenating text-string outputs from each image into one master text-string
        combined_text = ("\n**" + file_name + " **\n").join([combined_text, text])
        
        ## Creating separate text files for each image
        # with open(output_folder_path + "/" + file_name +'.txt', "w",encoding="utf8") as f:
        #     f.write(text)
        
        ## Writing in a pandas dataframe
        # df.loc[len(df.index)] = [img, output_folder_path + file_name +'.txt', language]
        
    ## Open a file with access mode 'a'
    file_object2 = open(output_folder_path + "/" + 'Combined_text.txt', 'a',encoding="utf8")
    file_object2.write(combined_text)
    file_object2.close()

In [ ]:
def doc_docx_xlsx_to_text(file_path):
    text_file = textract.process(file_paths)
    with open(os.path.split(file_paths)[0] + '/'+'text.txt', "w") as f:
        f.write(text_file.decode('utf8', 'strict'))

In [161]:
def raw_data_to_text_file(input1, language ='fra'):
    '''
    This function takes pdf, doc, docx and excel files and extract text file.

    Input parameters- 
        input1 is folder path containing documents.
        language of the input document. By default, it is french. eg: 'fra', 'ita', ..
    Output-
        For pdf files the output will be jpeg files and txt files
        For doc, docx and the excel file output will be only txt files.
    '''

    path = glob.glob(input1)
    print("List of folders present in input folder: ",path)

    for i in path:
        try:
            file_paths = str(glob.glob(i + '/*')[0])
            print("Input file path: ",file_paths)
        
            file_name_extn = os.path.basename(file_paths).split('.')[1]

            if file_name_extn == 'pdf':
                pdf_to_img(file_paths)
                img_to_txt(i,language)

            else: ##file_name in ['doc','docx','xlsx']:
                text_file = textract.process(file_paths)

                with open(os.path.split(file_paths)[0] + '/'+'Combined_text.txt', "w") as f:
                    f.write(text_file.decode('utf8', 'strict'))
        except:
            print('***files not found***')

In [ ]:
help(raw_data_to_text_file)

##**Testing**

In [162]:
raw_data_to_text_file('/content/drive/MyDrive/Zenon_POC_OCR/documents/*')

List of folders present in input folder:  ['/content/drive/MyDrive/Zenon_POC_OCR/documents/f1_doc', '/content/drive/MyDrive/Zenon_POC_OCR/documents/f1']
Input file path:  /content/drive/MyDrive/Zenon_POC_OCR/documents/f1_doc/f1_doc.doc
Input file path:  /content/drive/MyDrive/Zenon_POC_OCR/documents/f1/f1.pdf
Extracting text from f1
6
['/content/drive/MyDrive/Zenon_POC_OCR/documents/f1/1.jpeg', '/content/drive/MyDrive/Zenon_POC_OCR/documents/f1/2.jpeg', '/content/drive/MyDrive/Zenon_POC_OCR/documents/f1/3.jpeg', '/content/drive/MyDrive/Zenon_POC_OCR/documents/f1/4.jpeg', '/content/drive/MyDrive/Zenon_POC_OCR/documents/f1/5.jpeg', '/content/drive/MyDrive/Zenon_POC_OCR/documents/f1/6.jpeg']
File name: 1
File name: 2
File name: 3
File name: 4
File name: 5
File name: 6
